In [1]:
from transformers.testing_utils import torch_device

from slr.models.CorrNet import CorrNet
from slr.datasets.Phoenix2014DataModule import Phoenix2014DataModule
import numpy as np
import os
from slr.models.utils.decode import Decode

In [2]:
# load model
model = CorrNet.load_from_checkpoint(
    "/new_home/xzj23/workspace/SLR/experiments/Phoenix2014/CorrNet/2024-09-23_12-48-46/checkpoints/epoch=21-DEV_WER=19.10.ckpt")
model

CorrNet(
  (conv2d): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
        (bn1): Batc

In [3]:
# get data

with open(os.path.join("/new_home/xzj23/workspace/SLR/data/global_files/gloss_dict/phoenix2014_gloss_dict.npy"),
          'rb') as f:
    gloss_dict = np.load(f, allow_pickle=True).item()
datamodule = Phoenix2014DataModule(
    features_path="/new_home/xzj23/workspace/SLR/data/phoenix2014/phoenix-2014-multisigner/features/fullFrame-256x256px",
    annotations_path="/new_home/xzj23/workspace/SLR/data/phoenix2014/phoenix-2014-multisigner/annotations/manual",
    gloss_dict=gloss_dict,
    batch_size=2, num_workers=10
)
datamodule.setup(stage="fit")

batch = datamodule.train_dataloader().__iter__().__next__()
x, x_lgt, y, y_lgt, info = batch

In [4]:
x.shape

torch.Size([2, 232, 3, 224, 224])

In [5]:
y

tensor([ 951, 1120, 1134, 1120,  810,  142,  526,  396,    8,  569,  569,  569,
         569,  569, 1149, 1063,  719, 1149,  723,  565,  253,  465,  465,   20,
         132, 1195, 1113,  906])

In [6]:
x_lgt

tensor([232, 108])

In [7]:
y_lgt

tensor([18, 10])

In [8]:
info

[folder            31January_2011_Monday_heute_default-1/1/*.png
 signer                                                 Signer07
 annotation    SUED WETTER WINTER WETTER RUHIG BLEIBEN LANG H...
 Name: 31January_2011_Monday_heute_default-1, dtype: object,
 folder        04October_2012_Thursday_tagesschau_default-12/...
 signer                                                 Signer05
 annotation    NORDRAUM MAXIMAL ELF IX IX ACHTZEHN BIS ZWEIZW...
 Name: 04October_2012_Thursday_tagesschau_default-12, dtype: object]

In [9]:
# to device
device = "cuda:1"
model = model.to(device)
x = x.to(device)
y = y.to(device)
x_lgt = x_lgt.to(device)
y_lgt = y_lgt.to(device)

# forward
model.eval()
conv_logits, y_hat_logits, y_hat_lgt = model(x, x_lgt)

In [10]:
conv_logits.shape

torch.Size([55, 2, 1296])

In [11]:
y_hat_logits.shape

torch.Size([55, 2, 1296])

In [12]:
y_hat_lgt.shape

torch.Size([2])

In [13]:
# set decoder
decoder = Decode(gloss_dict=gloss_dict, num_classes=1296, search_mode='beam_search')

In [14]:
sentence = decoder.decode(y_hat_logits.softmax(dim=-1).cpu(), y_hat_lgt.cpu(), batch_first=False, probs=True)

In [15]:
for s in sentence:
    print(" ".join([t[0] for t in s]))

SUED WETTER WINTER WETTER RUHIG BLEIBEN LANG ABER MEHR VON NORD WOLKE
NORDRAUM MAXIMAL ELF SUED IX ACHTZEHN BIS ZWEIZWANZIG WENN SONNE


In [16]:
import torch
torch.cuda.empty_cache()

In [17]:
dddd = model.predict_step((x, x_lgt, y, y_lgt,info),0)

OutOfMemoryError: CUDA out of memory. Tried to allocate 356.00 MiB (GPU 1; 23.69 GiB total capacity; 22.83 GiB already allocated; 73.94 MiB free; 23.23 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
model = model.to("cpu")
ddd = model.predict_step(batch,0)

In [19]:
ddd

[[('SUED', 0),
  ('WETTER', 1),
  ('WINTER', 2),
  ('WETTER', 3),
  ('RUHIG', 4),
  ('BLEIBEN', 5),
  ('LANG', 6),
  ('ABER', 7),
  ('MEHR', 8),
  ('VON', 9),
  ('NORD', 10),
  ('WOLKE', 11)],
 [('NORDRAUM', 0),
  ('MAXIMAL', 1),
  ('ELF', 2),
  ('SUED', 3),
  ('IX', 4),
  ('ACHTZEHN', 5),
  ('BIS', 6),
  ('ZWEIZWANZIG', 7),
  ('WENN', 8),
  ('SONNE', 9)]]